# Make your own palaeo-agegrid globe(s)!

Agegrids should be for the masses!



In [15]:
import pygmt
import gplately
import gplately.pygplates as pygplates
import numpy as np
import os

Set up locations to files. 

This time we're going to plot some plate boundaries, so also need to set up filepaths for the rotation file and topologies (e.g. from GPlates 2.3 GeoData) 

In [10]:
plate_model_dir = '/Applications/GPlates_2.3.0/GeoData/FeatureCollections'

# specify files we need
rotation_filenames = '%s/Rotations/Muller2019-Young2019-Cao2020_CombinedRotations.rot' % (plate_model_dir)
rotation_model = pygplates.RotationModel(rotation_filenames)

topology_filenames = ['%s/DynamicPolygons/Muller2019-Young2019-Cao2020_PlateBoundaries.gpmlz'  % plate_model_dir, \
    '%s/DeformingLithosphere/Muller2019-Young2019-Cao2020_ActiveDeformation.gpmlz' % plate_model_dir, \
    '%s/DeformingLithosphere/Muller2019-Young2019-Cao2020_InactiveDeformation.gpmlz' % plate_model_dir ]

topology_features = pygplates.FeatureCollection()
for file in topology_filenames:
    topology_feature = pygplates.FeatureCollection(file)
    topology_features.add(topology_feature)

# don't actually need these, but specified just incase..
static_polygon_filename = '%s/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons.gpmlz' % (plate_model_dir)
 
coastline_filename = '%s/Coastlines/Global_EarthByte_GPlates_PresentDay_Coastlines.gpmlz' % (plate_model_dir)
coastlines = pygplates.FeatureCollection(coastline_filename)

cob_filename = '%s/COBs/Muller2019-Young2019-Cao2020_COBs.gpmlz' % (plate_model_dir)

# create a gplately PlateReconstruction
gplates23_model = gplately.PlateReconstruction(rotation_model, topology_features=topology_features, 
                                               static_polygons=static_polygon_filename)

In [37]:
# --- agegrids
agegrid_dir = '/Users/nickywright/Downloads/Muller2019-Young2019-Cao2020_Agegrids/Muller2019-Young2019-Cao2020_netCDF/'
agegrid_filename = 'Muller2019-Young2019-Cao2020_AgeGrid-'
agegrid_filename_ext = '.nc'

## Plot!

Using [pygmt](https://www.pygmt.org/latest/index.html) or your favourite plotting program.

Note: we **don't** want any edges on this, because no one likes ugly borders through the Pacific Ocean.
We will also use `gplately` to help us plot plate boundaries!

In [20]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')

In [22]:
# test getting plate boundaries at X time...
age = 50

gplot = gplately.PlotTopologies(gplates23_model, coastlines=coastlines, COBs=cob_filename, time=age)
gdf_coastlines = gplot.get_coastlines()
gdf_cobs = gplot.get_continent_ocean_boundaries()

gdf_subduction_left, gdf_subduction_right = gplot.get_subduction_direction()
gdf_ridges_transforms = gplot.get_ridges_and_transforms()
gdf_ridges = gplot.get_ridges()
gdf_transforms = gplot.get_transforms()
gdf_fracturezones = gplot.get_fracture_zones()
gdf_ipb = gplot.get_inferred_paleo_boundaries()
gdf_sutures = gplot.get_sutures()
gdf_misc = gplot.get_misc_boundaries()

gdf_topo = gplot.get_all_topologies()
gdf_topo_plates = gdf_topo[(gdf_topo['feature_name'] == 'TopologicalClosedPlateBoundary')]


In [64]:
# gdf_topo_plates

In [65]:
# gdf_topo_plates.plot()

Going to end up with a line in the middle of the Pacific using these :( 

Perhaps we can convert to lines, and then get rid of the -180° line? To do...

In [55]:
def plot_paleo_agegrid(age):
    gplot = gplately.PlotTopologies(gplates23_model, coastlines=coastlines, COBs=cob_filename, time=age)
    gdf_coastlines = gplot.get_coastlines()
    # gdf_cobs = gplot.get_continent_ocean_boundaries()

    gdf_subduction_left, gdf_subduction_right = gplot.get_subduction_direction()
    gdf_ridges_transforms = gplot.get_ridges_and_transforms()
    # gdf_ridges = gplot.get_ridges()
    # gdf_transforms = gplot.get_transforms()
    gdf_fracturezones = gplot.get_fracture_zones()
    gdf_ipb = gplot.get_inferred_paleo_boundaries()
    gdf_sutures = gplot.get_sutures()
    gdf_misc = gplot.get_misc_boundaries()

    gdf_topo = gplot.get_all_topologies()
    gdf_topo_plates = gdf_topo[(gdf_topo['feature_name'] == 'TopologicalClosedPlateBoundary')]
    
    # ----- parameters for plot
    region = 'd'
    width = '18/9'
    projection = 'X'

    # plate boundary stuff
    plateboundary_width = '0.5p'
    plate_colour = 'white'
    subduction_zone_colour = 'white'
    ridge_colour = 'white'

    age_cpt = '/Users/nickywright/Data/Age/Seton++_2020/age_2020.cpt'

    output_filename = 'tmp/age_%s.png' % age
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ------ plot
    fig = pygmt.Figure()

    fig.grdimage(grid='%s/%s%s%s' % (agegrid_dir, agegrid_filename, age, agegrid_filename_ext),
                 region=region, projection="%s%sc" % (projection, width), cmap=age_cpt, dpi=1000) # this dpi is completely overkill. You could reduce this...

    # fig.plot(data=gdf_topo_plates.geometry, pen='%s,%s' % (plateboundary_width, plate_colour))
    fig.plot(data=gdf_ridges_transforms, pen='%s,%s' % (plateboundary_width, ridge_colour))
    fig.plot(data=gdf_ipb, pen='%s,%s' % (plateboundary_width, plate_colour))
    # fig.plot(data=gdf_misc, pen='%s,%s' % (plateboundary_width, plate_colour))
    fig.plot(data=gdf_fracturezones, pen='%s,%s' % (plateboundary_width, plate_colour))
    fig.plot(data=gdf_subduction_left, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+l+t')
    fig.plot(data=gdf_subduction_right, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+r+t')
    fig.plot(data=gdf_coastlines, fill='black')

    # fig.show(width=1000)
    fig.savefig(output_filename, dpi=1000) # this dpi is completely overkill. You could reduce this...
    
    return

In [61]:
recon_times = np.arange(0, 251, 1)

In [60]:
for time in recon_times:
    plot_paleo_agegrid(time)

/Users/nickywright/miniconda3/lib/python3.9/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/nickywright/miniconda3/lib/python3.9/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/nickywright/miniconda3/lib/python3.9/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/nickywright/miniconda3/lib/python3.9/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_f

---
# Create dymaxion globe through time!

Since there is this handy python package [`pydymax`](https://github.com/Teque5/pydymax) that will convert things into a dymaxion globe for you, we can actually do this through time!

In [7]:
import dymax

In [100]:
# Warning: this really IS NOT quick since my images are giant.
# It would probably be faster if I didn't use a ridiculous dpi

recon_times = np.arange(100, 101, 1)

for time in recon_times:
    input_filename = 'tmp/age_%s.png' % time
    output_filename = 'tmp/age_dymax-%s.png' % time
    
    dymax.convert_rectimage_2_dymaximage(input_filename, output_filename, output_filename, scale=1200, save=True, show=False)

:: input image resolution = (7087, 3544)
:: output image resolution = (6600, 3120)
:: sweeping over Longitudes:
-180.00 -178.98 -177.97 -176.95 -175.94 -174.92 -173.90 -172.89 -171.87 -170.86 -169.84 -168.82 -167.81 -166.79 -165.77 -164.76 -163.74 -162.73 -161.71 -160.69 -159.68 -158.66 -157.65 -156.63 -155.61 -154.60 -153.58 -152.57 -151.55 -150.53 -149.52 -148.50 -147.49 -146.47 -145.45 -144.44 -143.42 -142.40 -141.39 -140.37 -139.36 -138.34 -137.32 -136.31 -135.29 -134.28 -133.26 -132.24 -131.23 -130.21 -129.20 -128.18 -127.16 -126.15 -125.13 -124.12 -123.10 -122.08 -121.07 -120.05 -119.03 -118.02 -117.00 -115.99 -114.97 -113.95 -112.94 -111.92 -110.91 -109.89 -108.87 -107.86 -106.84 -105.83 -104.81 -103.79 -102.78 -101.76 -100.75 -099.73 -098.71 -097.70 -096.68 -095.66 -094.65 -093.63 -092.62 -091.60 -090.58 -089.57 -088.55 -087.54 -086.52 -085.50 -084.49 -083.47 -082.46 -081.44 -080.42 -079.41 -078.39 -077.38 -076.36 -075.34 -074.33 -073.31 -072.29 -071.28 -070.26 -069.25 -068.23 

### Create an animation!

This uses ffmpeg. I do **not** understand how to get ffmpeg filters to work sometimes, but this seems to go ok...

In [114]:
%%bash

cp tmp/age_dymax-0.png tmp/age_dymax0.png

# ffmpeg -r 6 -start_number -250 -i tmp/age_dymax%0d.png -c:v prores -pix_fmt yuva444p10le -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -y dymax_animation.mov
ffmpeg -r 10 -start_number -250 -i tmp/age_dymax%0d.png -pix_fmt yuv420p -vf "split=2[clr][bg];[bg]drawbox=c=white:t=fill[bg];[bg][clr]overlay,scale=trunc(iw/8)*2:trunc(ih/8)*2,drawtext=fontsize=70:fontfile=/System/Library/Fonts/Avenir.ttc:text='%{eif\:250-n\:d} Ma':x=40: y=(40): fontcolor=black" -y dymax_animation.mp4

# scale to 1/3 size: "scale=trunc(iw/6)*2:trunc(ih/6)*2"
# scale to 1/2 size "scale=trunc(iw/4)*2:trunc(ih/4)*2" 
 

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [115]:
%%html

<video width="800" height=" " 
       src="dymax_animation.mp4"  
       controls>
</video>